In [1]:
# Dependencies installation
%pip install opencv-python-headless
%pip install pillow
%pip install ultralytics
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install tensorflow
%pip install paddlepaddle "paddleocr>=2.0.1"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-5.29.2-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.2-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.2
    Uninstalling protobuf-3.20.2:
      Successfully uninstalled protobuf-3.20.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlepaddle 2.6.2 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 5.29.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-3.20.2-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.2-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports
from ultralytics import YOLO
import os
import cv2
import json
import numpy as np
from pathlib import Path
import random
import shutil
import csv
from paddleocr import PaddleOCR

In [ ]:
# Constants
CURRENT_PATH = os.getcwd()
CSV_FILE = os.path.join(CURRENT_PATH, 'WaterMeters', 'data.csv')
LABELS_DIR = os.path.join(CURRENT_PATH, 'etiquetas')
CLASS_ID = 0

In [3]:
# Crear directorio de salida si no existe
os.makedirs(LABELS_DIR, exist_ok=True)

# Leer el archivo CSV
with open(CSV_FILE, 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        image_name = row[0].split('.')[0]  # Obtener el nombre sin extensión
        
        # Reemplazar comillas simples por comillas dobles para formato JSON válido
        try:
            fixed_data = row[2].replace("'", '"')
            polygon_data = json.loads(fixed_data)['data']
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error al procesar la fila {row[0]}: {e}")
            continue

        # Extraer coordenadas normalizadas
        coords = []
        for point in polygon_data:
            coords.append(str(point['x']))
            coords.append(str(point['y']))

        # Crear el archivo de etiqueta
        output_path = os.path.join(LABELS_DIR, f"{image_name}.txt")
        with open(output_path, 'w') as out_file:
            out_file.write(f"{CLASS_ID} " + ' '.join(coords))

print(f"Conversión completada. Archivos guardados en: {LABELS_DIR}")

NameError: name 'LABELS_DIR' is not defined

In [ ]:
source_images =  os.path.join(CURRENT_PATH, 'WaterMeters/images')
source_labels =  os.path.join(CURRENT_PATH, 'etiquetas')  # Ajusta según tu ruta de etiquetas
output_dir = os.path.join(CURRENT_PATH, 'data')  # Salida de los datos organizados

# Directorios para train, val y test
train_images = os.path.join(output_dir, 'images/train')
val_images = os.path.join(output_dir, 'images/val')
test_images = os.path.join(output_dir, 'images/test')
train_labels = os.path.join(output_dir, 'labels/train')
val_labels = os.path.join(output_dir, 'labels/val')
test_labels = os.path.join(output_dir, 'labels/test')

# Crear directorios si no existen
os.makedirs(train_images, exist_ok=True)
os.makedirs(val_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)
os.makedirs(train_labels, exist_ok=True)
os.makedirs(val_labels, exist_ok=True)
os.makedirs(test_labels, exist_ok=True)

# Listar las imágenes (.jpg) y verificar si existen
image_files = [f for f in os.listdir(source_images) if f.endswith('.jpg')]

# Si no hay imágenes en el directorio de origen, imprimir un error
if len(image_files) == 0:
    print("Error: No se encontraron imágenes en la carpeta de origen.")
else:
    # Dividir las imágenes en 80% para entrenamiento, 10% para validación y 10% para prueba
    random.shuffle(image_files)
    train_files = image_files[:int(0.8 * len(image_files))]
    val_files = image_files[int(0.8 * len(image_files)):int(0.9 * len(image_files))]
    test_files = image_files[int(0.9 * len(image_files)):]

    # Mover las imágenes y sus etiquetas correspondientes
    for file in train_files:
        img_src = os.path.join(source_images, file)
        img_dst = os.path.join(train_images, file)
        label_src = os.path.join(source_labels, file.replace('.jpg', '.txt'))
        label_dst = os.path.join(train_labels, file.replace('.jpg', '.txt'))

        # Verificar si la imagen y la etiqueta existen antes de copiar
        if os.path.exists(img_src) and os.path.exists(label_src):
            shutil.copy(img_src, img_dst)  # Copiar imagen
            shutil.copy(label_src, label_dst)  # Copiar etiqueta
        else:
            print(f"Advertencia: No se encontró la etiqueta para {file} o la imagen no existe.")

    for file in val_files:
        img_src = os.path.join(source_images, file)
        img_dst = os.path.join(val_images, file)
        label_src = os.path.join(source_labels, file.replace('.jpg', '.txt'))
        label_dst = os.path.join(val_labels, file.replace('.jpg', '.txt'))

        # Verificar si la imagen y la etiqueta existen antes de copiar
        if os.path.exists(img_src) and os.path.exists(label_src):
            shutil.copy(img_src, img_dst)  # Copiar imagen
            shutil.copy(label_src, label_dst)  # Copiar etiqueta
        else:
            print(f"Advertencia: No se encontró la etiqueta para {file} o la imagen no existe.")

    for file in test_files:
        img_src = os.path.join(source_images, file)
        img_dst = os.path.join(test_images, file)
        label_src = os.path.join(source_labels, file.replace('.jpg', '.txt'))
        label_dst = os.path.join(test_labels, file.replace('.jpg', '.txt'))

        # Verificar si la imagen y la etiqueta existen antes de copiar
        if os.path.exists(img_src) and os.path.exists(label_src):
            shutil.copy(img_src, img_dst)  # Copiar imagen
            shutil.copy(label_src, label_dst)  # Copiar etiqueta
        else:
            print(f"Advertencia: No se encontró la etiqueta para {file} o la imagen no existe.")

    print("Imágenes y etiquetas divididas correctamente en train, val y test.")

Imágenes y etiquetas divididas correctamente en train, val y test.


In [51]:
output_dir = Path(CURRENT_PATH) / 'data' / 'data.yaml'
train = Path(CURRENT_PATH) / 'data' / 'images' / 'train'
val = Path(CURRENT_PATH) / 'data' / 'images' / 'val'
test = Path(CURRENT_PATH) / 'data' / 'images' / 'test'

# Definir el archivo de configuración
data_yaml = f"""train: {train}
val: {val}
test: {test}

nc: 1
names:
  ['medidor']  # Nombres de las clases"""

# Crear el directorio si no existe
os.makedirs(output_dir.parent, exist_ok=True)

# Guardar el archivo YAML
with open(output_dir, 'w') as f:
    f.write(data_yaml)

print(f"Archivo de configuración 'data.yaml' creado en: {output_dir}")


Archivo de configuración 'data.yaml' creado en: c:\Users\HP\Desktop\PM\data\data.yaml


In [ ]:
# Directorio del archivo YAML
data_dir = Path(CURRENT_PATH) / 'data' / 'data.yaml'

# Ruta del modelo, asegúrate de que el archivo esté en el directorio correcto
model_path = Path(CURRENT_PATH) / 'yolo11n-seg.pt'

# Cargar el modelo YOLOv11 de segmentación
model = YOLO(model_path)

# Continuar desde el último checkpoint
train_results = model.train(
    data=data_dir,         # Directorio con el archivo YAML
    epochs=50,             # Número de épocas
    imgsz=640,             # Tamaño de las imágenes
    batch=16,              # Tamaño del batch
    task='segmentation'    # Especificar que es una tarea de segmentación
)

New https://pypi.org/project/ultralytics/8.3.48 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.47  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx)
engine\trainer: task=segment, mode=train, model=c:\Users\HP\Desktop\PM\yolo11n-seg.pt, data=c:\Users\HP\Desktop\PM\data\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

train: Scanning C:\Users\HP\Desktop\PM\data\labels\train... 880 images, 0 backgrounds, 0 corrupt: 100%|██████████| 880/880 [00:04<00:00, 196.97it/s]


train: New cache created: C:\Users\HP\Desktop\PM\data\labels\train.cache


val: Scanning C:\Users\HP\Desktop\PM\data\labels\val... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<00:00, 182.50it/s]


val: New cache created: C:\Users\HP\Desktop\PM\data\labels\val.cache
Plotting labels to runs\segment\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50         0G     0.3896     0.4237     0.2941     0.7934         16        640: 100%|██████████| 55/55 [16:12<00:00, 17.68s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:38<00:00,  9.74s/it]

                   all        110        110          1      0.997      0.995      0.924          1      0.997      0.995      0.895



50 epochs completed in 14.853 hours.
Optimizer stripped from runs\segment\train\weights\last.pt, 6.0MB
Optimizer stripped from runs\segment\train\weights\best.pt, 6.0MB

Validating runs\segment\train\weights\best.pt...
Ultralytics 8.3.47  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:36<00:00,  9.03s/it]


                   all        110        110          1      0.997      0.995      0.924          1      0.997      0.995      0.895
Speed: 2.7ms preprocess, 266.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\segment\train


In [5]:
CURRENT_PATH = os.getcwd()

# Directorio del archivo YAML
data_dir = Path(CURRENT_PATH) / 'data' / 'data.yaml'

# Ruta del modelo, asegúrate de que el archivo esté en el directorio correcto
model_path = Path(CURRENT_PATH) / 'runs' / 'segment' / 'train' / 'weights' / 'best.pt'

# Cargar el modelo YOLOv11 de segmentación
model = YOLO(model_path)

# Continuar desde el último checkpoint
val_results = model.val(
    data=data_dir,         # Directorio con el archivo YAML
    epochs=50,             # Número de épocas
    imgsz=640,             # Tamaño de las imágenes
    batch=16,              # Tamaño del batch
    task='segmentation'    # Especificar que es una tarea de segmentación
)

print("Resultados de Validación:")
print(val_results)

Ultralytics 8.3.47  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning C:\Users\HP\Desktop\PM\data\labels\val.cache... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:25<00:00,  3.68s/it]


                   all        110        110          1      0.997      0.995      0.924          1      0.997      0.995      0.894
Speed: 3.0ms preprocess, 172.7ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\segmentation\val
Resultados de Validación:
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DE65D81640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.0

In [7]:
from ultralytics import YOLO
from pathlib import Path

# Directorio del archivo YAML
data_dir = Path(CURRENT_PATH) / 'data' / 'data.yaml'

# Ruta del modelo entrenado
model_path = Path(CURRENT_PATH) / 'runs' / 'segment' / 'train' / 'weights' / 'best.pt'

# Cargar el modelo entrenado
model = YOLO(model_path)

# Realizar la prueba
test_results = model.val(
    data=data_dir,       # Directorio con el archivo YAML
    imgsz=640,           # Tamaño de las imágenes
    task='segmentation', # Especificar que es una tarea de segmentación
    split='test'
)

# Mostrar los resultados de la prueba
print("Resultados del Test:")
print(test_results)


Ultralytics 8.3.47  Python-3.12.6 torch-2.5.1+cpu CPU (AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning C:\Users\HP\Desktop\PM\data\labels\test... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<00:00, 152.01it/s]

val: New cache created: C:\Users\HP\Desktop\PM\data\labels\test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:29<00:00,  4.19s/it]


                   all        111        111          1      0.991      0.995      0.925          1      0.991      0.995      0.885
Speed: 6.0ms preprocess, 192.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\segmentation\val2
Resultados del Test:
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DE697AD9D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002

In [168]:
import numpy as np

model_path = os.path.join('runs', 'segment', 'train', 'weights', 'best.pt')

# Cargar el modelo entrenado
model = YOLO(model_path)

# Cargar una imagen
image_path = r'data\images\test\id_319_value_1669_853.jpg'
image = cv2.imread(image_path)

# Realizar la predicción
results = model(image, save=True, conf=0.5)  # 'conf' es el umbral de confianza

# Mostrar la imagen con los resultados
for result in results:
    annotated_frame = result.plot()  # Dibuja las predicciones en la imagen
    cv2.imshow("Predicción", annotated_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


0: 640x480 1 medidor, 230.0ms
Speed: 5.1ms preprocess, 230.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\segment\predict4


In [167]:
import os
import cv2
import numpy as np
import re
from paddleocr import PaddleOCR

# Definir la ruta del modelo
model_path = os.path.join('runs', 'segment', 'train', 'weights', 'best.pt')

# Cargar el modelo entrenado
model = YOLO(model_path)

# Cargar una imagen
image_path = r'data\images\test\id_319_value_1669_853.jpg'
image = cv2.imread(image_path)

# Verificar que la imagen se haya cargado correctamente
if image is None:
    raise FileNotFoundError(f"No se pudo cargar la imagen en la ruta: {image_path}")

# Realizar la predicción con YOLO
results = model(image, conf=0.5)  # 'conf' es el umbral de confianza

# Inicializar el modelo PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='es')

# Cadena para concatenar los resultados reconocidos
numerical_results = ""

# Procesar cada resultado para recortar la zona de interés
for result in results:
    # Obtener las máscaras (zonas de interés)
    masks = result.masks

    if masks:
        for i, mask in enumerate(masks.data):
            # Convertir la máscara a una imagen binaria
            mask = mask.cpu().numpy()
            mask_binary = (mask * 255).astype(np.uint8)

            # Redimensionar la máscara para que coincida con el tamaño de la imagen original
            mask_resized = cv2.resize(mask_binary, (image.shape[1], image.shape[0]))

            # Crear una máscara de 3 canales para aplicarla a la imagen original
            mask_3channel = cv2.merge([mask_resized, mask_resized, mask_resized])

            # Aplicar la máscara a la imagen original para extraer la región de interés exacta
            masked_image = cv2.bitwise_and(image, mask_3channel)

            # Encontrar el bounding box de la región cubierta por la máscara
            x, y, w, h = cv2.boundingRect(mask_resized)

            # Recortar la región de interés de la imagen original usando el bounding box
            roi = masked_image[y:y+h, x:x+w]

            # Convertir a escala de grises
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

            # Ajustar el contraste y brillo de la imagen en escala de grises
            alpha = 1.25  # Factor de contraste (1.0 significa sin cambio)
            beta = 2  # Valor de brillo (0 significa sin cambio)

            # Aplicar el ajuste de contraste y brillo
            adjusted_gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

            # Realizar la OCR en la imagen ajustada
            ocr_result = ocr.ocr(adjusted_gray, cls=True)
            for line in ocr_result[0]:
                text = line[1][0]  # Obtener el texto reconocido
                print(f"Texto reconocido: {text}")
                numerical_results += text

            # Guardar la imagen ajustada
            image_path2 = f'image{i}.jpg'
            roi_path = f'roi{i}.jpg'
            adjusted_roi_path = f'adjusted_roi_gray_{i}.jpg'
            cv2.imwrite(adjusted_roi_path, adjusted_gray)
            cv2.imwrite(roi_path, roi)
            cv2.imwrite(image_path2, image)
            print(f"Zona de interés procesada guardada en: {adjusted_roi_path}")

# Imprimir los resultados concatenados
print("Resultados:", numerical_results.strip())



0: 640x480 1 medidor, 209.1ms
Speed: 6.0ms preprocess, 209.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 480)
[2024/12/11 22:24:11] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\HP/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\User

In [ ]:
import os
import cv2
import numpy as np
import re
from paddleocr import PaddleOCR
from tqdm import tqdm 

# Definir la ruta del modelo
model_path = os.path.join('runs', 'segment', 'train', 'weights', 'best.pt')

# Cargar el modelo entrenado
model = YOLO(model_path)

# Directorio de imágenes
image_dir = os.path.join('data', 'images', 'test')

# Directorio para guardar las imágenes recortadas
cropped_dir = os.path.join('data', 'images', 'cropped')
os.makedirs(cropped_dir, exist_ok=True)

# Inicializar el modelo PaddleOCR con parámetros ajustados
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang='en',
)

# Ruta del archivo de resultados
result_file = "resultados 20AMP EN TODO PRUEBA.txt"

# Limpiar o crear el archivo de resultados
with open(result_file, "w") as f:
    f.write("Resultados de OCR:\n\n")

def extract_value_from_filename(filename):
    """Extrae el valor compuesto por las dos partes después de 'value_' en el nombre del archivo."""
    match = re.search(r'value_(\d+)_(\d+)', filename)
    if match:
        return match.group(1) + match.group(2)  # Combinar ambos grupos como un único número
    return None

def calculate_match_percentage(ocr_result, expected_value):
    """Calcula el porcentaje de coincidencia basado en caracteres correctos detectados, sin importar el orden."""
    ocr_chars = list(ocr_result)
    expected_chars = list(expected_value)
    correct_chars = 0

    # Contar los caracteres correctos (sin importar el orden)
    for char in expected_chars:
        if char in ocr_chars:
            correct_chars += 1
            ocr_chars.remove(char)

    total_chars = len(expected_chars)
    if total_chars == 0:
        return 0

    match_percentage = (correct_chars / total_chars) * 100
    return match_percentage

# Variables para calcular el porcentaje global
total_images = 0
successful_matches = 0
deficient_matches = 0
none_type_errors = 0

# Factor de ampliación del recorte
expansion_factor = 0.2

# Procesar las imágenes en el directorio
for filename in tqdm(os.listdir(image_dir)):
    image_path = os.path.join(image_dir, filename)

    # Verificar que sea una imagen
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        # Cargar la imagen
        image = cv2.imread(image_path)
        if image is None:
            print(f"No se pudo cargar la imagen: {image_path}")
            none_type_errors += 1
            continue

        # Obtener el valor esperado del nombre del archivo
        expected_value = extract_value_from_filename(filename)
        if expected_value is None:
            print(f"No se encontró un valor esperado en el archivo: {filename}")
            none_type_errors += 1
            continue

        # Realizar la predicción con YOLO
        results = model(image, conf=0.5)
        numerical_results = ""

        # Procesar cada resultado para recortar la zona de interés
        for result in results:
            masks = result.masks
            if masks:
                for i, mask in enumerate(masks.data):
                    mask = mask.cpu().numpy()
                    mask_binary = (mask * 255).astype(np.uint8)
                    mask_resized = cv2.resize(mask_binary, (image.shape[1], image.shape[0]))
                    mask_3channel = cv2.merge([mask_resized, mask_resized, mask_resized])
                    masked_image = cv2.bitwise_and(image, mask_3channel)

                    # Obtener el rectángulo delimitador y expandirlo
                    x, y, w, h = cv2.boundingRect(mask_resized)
                    
                    # Calcular el nuevo recorte con el factor de ampliación
                    x_exp = max(int(x - w * expansion_factor), 0)
                    y_exp = max(int(y - h * expansion_factor), 0)
                    w_exp = min(int(w + w * 2 * expansion_factor), image.shape[1] - x_exp)
                    h_exp = min(int(h + h * 2 * expansion_factor), image.shape[0] - y_exp)
                    
                    roi = image[y_exp:y_exp + h_exp, x_exp:x_exp + w_exp]

                    # Guardar la imagen recortada
                    cropped_filename = f"{os.path.splitext(filename)[0]}_mask{i}.png"
                    cropped_path = os.path.join(cropped_dir, cropped_filename)

                    # Realizar OCR
                    ocr_result = ocr.ocr(roi)
                    for line in ocr_result[0]:
                        text = line[1][0]
                        numerical_results += text

                    # Limpiar el resultado OCR
                    ocr_cleaned = re.sub(r'\D', '', numerical_results)  # Eliminar caracteres no numéricos

                    # Añadir el resultado OCR en un cuadro de texto en la imagen
                    if ocr_cleaned:
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        font_scale = 0.5
                        font_color = (0, 255, 0)  # Verde
                        thickness = 2
                        text_position = (10, 30)  # Coordenadas para el texto

                        # Añadir un rectángulo para el fondo del texto
                        (text_width, text_height), baseline = cv2.getTextSize(ocr_cleaned, font, font_scale, thickness)
                        cv2.rectangle(roi, (text_position[0] - 5, text_position[1] - text_height - 5),
                                      (text_position[0] + text_width + 5, text_position[1] + baseline + 5),
                                      (0, 0, 0), cv2.FILLED)

                        # Poner el texto en la imagen
                        cv2.putText(roi, ocr_cleaned, text_position, font, font_scale, font_color, thickness)

                    # Guardar la imagen recortada con el texto
                    cv2.imwrite(cropped_path, roi)

        # Calcular el porcentaje de coincidencia
        match_percentage = calculate_match_percentage(ocr_cleaned, expected_value)

        # Actualizar contadores globales
        total_images += 1
        if match_percentage >= 90 :
            successful_matches += 1
        else:
            deficient_matches += 1

        # Guardar el resultado en el archivo
        with open(result_file, "a") as f:
            f.write(f"Archivo: {filename}\n")
            f.write(f"Valor esperado: {expected_value}\n")
            f.write(f"Valor OCR: {numerical_results}\n")
            f.write(f"Porcentaje de coincidencia: {match_percentage:.2f}%\n\n")

    except Exception as e:
        none_type_errors += 1
        with open(result_file, "a") as f:
            f.write(f"Error procesando la imagen {filename}: {e}\n\n")

# Guardar el total de imágenes procesadas
with open(result_file, "a") as f:
    f.write(f"\nTotal de imágenes procesadas: {total_images}\n")
    f.write(f"Cantidad de lecturas correctas: {successful_matches}\n")
    f.write(f"Cantidad de lecturas deficientes: {deficient_matches}\n")
    f.write(f"Cantidad de errores NoneType: {none_type_errors}\n")

In [ ]:
import os
import cv2
import numpy as np
import re
from paddleocr import PaddleOCR
from tqdm import tqdm  # Para mostrar progreso

# Definir la ruta del modelo
model_path = os.path.join('runs', 'segment', 'train', 'weights', 'best.pt')

# Cargar el modelo entrenado
model = YOLO(model_path)

# Directorio de imágenes
image_dir = os.path.join('data', 'images', 'test')

# Directorio para guardar las imágenes recortadas y filtradas
cropped_dir = os.path.join('data', 'images', 'cropped')
filtered_dir = os.path.join('data', 'images', 'cropper_filter')
os.makedirs(cropped_dir, exist_ok=True)
os.makedirs(filtered_dir, exist_ok=True)

# Inicializar el modelo PaddleOCR con parámetros ajustados
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang='en',
)

# Ruta del archivo de resultados
result_file = "resultados_20AMP_EN_CLS_F_TODO.txt"

# Limpiar o crear el archivo de resultados
with open(result_file, "w") as f:
    f.write("Resultados de OCR:\n\n")

def extract_value_from_filename(filename):
    """Extrae el valor compuesto por las dos partes después de 'value_' en el nombre del archivo."""
    match = re.search(r'value_(\d+)_(\d+)', filename)
    if match:
        return match.group(1) + match.group(2)  # Combinar ambos grupos como un único número
    return None

def calculate_match_percentage(ocr_result, expected_value):
    """Calcula el porcentaje de coincidencia basado en caracteres correctos detectados, sin importar el orden."""
    ocr_chars = list(ocr_result)
    expected_chars = list(expected_value)
    correct_chars = 0

    # Contar los caracteres correctos (sin importar el orden)
    for char in expected_chars:
        if char in ocr_chars:
            correct_chars += 1
            ocr_chars.remove(char)

    total_chars = len(expected_chars)
    if total_chars == 0:
        return 0

    match_percentage = (correct_chars / total_chars) * 100
    return match_percentage

# Variables para calcular el porcentaje global
total_images = 0
successful_matches = 0
deficient_matches = 0
none_type_errors = 0

# Factor de ampliación del recorte
expansion_factor = 0.2

# Procesar las imágenes en el directorio
for filename in tqdm(os.listdir(image_dir)):
    image_path = os.path.join(image_dir, filename)

    # Verificar que sea una imagen
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        # Cargar la imagen
        image = cv2.imread(image_path)
        if image is None:
            print(f"No se pudo cargar la imagen: {image_path}")
            none_type_errors += 1
            continue

        # Obtener el valor esperado del nombre del archivo
        expected_value = extract_value_from_filename(filename)
        if expected_value is None:
            print(f"No se encontró un valor esperado en el archivo: {filename}")
            none_type_errors += 1
            continue

        # Realizar la predicción con YOLO
        results = model(image, conf=0.5)
        numerical_results = ""

        # Procesar cada resultado para recortar la zona de interés
        for result in results:
            masks = result.masks
            if masks:
                for i, mask in enumerate(masks.data):
                    mask = mask.cpu().numpy()
                    mask_binary = (mask * 255).astype(np.uint8)
                    mask_resized = cv2.resize(mask_binary, (image.shape[1], image.shape[0]))
                    mask_3channel = cv2.merge([mask_resized, mask_resized, mask_resized])
                    masked_image = cv2.bitwise_and(image, mask_3channel)

                    # Obtener el rectángulo delimitador y expandirlo
                    x, y, w, h = cv2.boundingRect(mask_resized)
                    
                    # Calcular el nuevo recorte con el factor de ampliación
                    x_exp = max(int(x - w * expansion_factor), 0)
                    y_exp = max(int(y - h * expansion_factor), 0)
                    w_exp = min(int(w + w * 2 * expansion_factor), image.shape[1] - x_exp)
                    h_exp = min(int(h + h * 2 * expansion_factor), image.shape[0] - y_exp)
                    
                    roi = image[y_exp:y_exp + h_exp, x_exp:x_exp + w_exp]

                    # Aplicar el filtro de contraste (1.25) y brillo (2)
                    contrast_alpha = 1.25  # Aumentar contraste
                    brightness_beta = 2   # Aumentar brillo
                    roi_filtered = cv2.convertScaleAbs(roi, alpha=contrast_alpha, beta=brightness_beta)

                    # Guardar la imagen filtrada
                    filtered_filename = f"{os.path.splitext(filename)[0]}_mask{i}_filtered.png"
                    filtered_path = os.path.join(filtered_dir, filtered_filename)
                    cv2.imwrite(filtered_path, roi_filtered)

                    # Realizar OCR
                    ocr_result = ocr.ocr(roi_filtered)
                    for line in ocr_result[0]:
                        text = line[1][0]
                        numerical_results += text

                    # Limpiar el resultado OCR
                    ocr_cleaned = re.sub(r'\D', '', numerical_results)  # Eliminar caracteres no numéricos

                    # Añadir el resultado OCR en un cuadro de texto en la imagen
                    if ocr_cleaned:
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        font_scale = 0.5
                        font_color = (0, 255, 0)  # Verde
                        thickness = 2
                        text_position = (10, 30)  # Coordenadas para el texto

                        # Añadir un rectángulo para el fondo del texto
                        (text_width, text_height), baseline = cv2.getTextSize(ocr_cleaned, font, font_scale, thickness)
                        cv2.rectangle(roi_filtered, (text_position[0] - 5, text_position[1] - text_height - 5),
                                      (text_position[0] + text_width + 5, text_position[1] + baseline + 5),
                                      (0, 0, 0), cv2.FILLED)

                        # Poner el texto en la imagen
                        cv2.putText(roi_filtered, ocr_cleaned, text_position, font, font_scale, font_color, thickness)

                    # Guardar la imagen recortada con el texto
                    cropped_filename = f"{os.path.splitext(filename)[0]}_mask{i}.png"
                    cropped_path = os.path.join(cropped_dir, cropped_filename)
                    cv2.imwrite(cropped_path, roi_filtered)

        # Calcular el porcentaje de coincidencia
        match_percentage = calculate_match_percentage(ocr_cleaned, expected_value)

        # Actualizar contadores globales
        total_images += 1
        if match_percentage >= 90 :
            successful_matches += 1
        else:
            deficient_matches += 1

        # Guardar el resultado en el archivo
        with open(result_file, "a") as f:
            f.write(f"Archivo: {filename}\n")
            f.write(f"Valor esperado: {expected_value}\n")
            f.write(f"Valor OCR: {ocr_cleaned}\n")
            f.write(f"Porcentaje de coincidencia: {match_percentage:.2f}%\n\n")

    except Exception as e:
        none_type_errors += 1
        with open(result_file, "a") as f:
            f.write(f"Error procesando la imagen {filename}: {e}\n\n")

# Guardar el total de imágenes procesadas
with open(result_file, "a") as f:
    f.write(f"\nTotal de imágenes procesadas: {total_images}\n")
    f.write(f"Cantidad de lecturas correctas: {successful_matches}\n")
    f.write(f"Cantidad de lecturas deficientes: {deficient_matches}\n")
    f.write(f"Cantidad de errores NoneType: {none_type_errors}\n")


In [ ]:
import os
import cv2
import numpy as np
import re
from paddleocr import PaddleOCR
from tqdm import tqdm
from ultralytics import YOLO  # Asegúrate de que YOLO esté correctamente importado

# Definir la ruta del modelo
model_path = os.path.join('runs', 'segment', 'train', 'weights', 'best.pt')

# Cargar el modelo entrenado
model = YOLO(model_path)

# Directorio de imágenes
image_dir = os.path.join('data', 'images', 'test')

# Directorio para guardar las imágenes recortadas
cropped_dir = os.path.join('data', 'images', 'cropped')
os.makedirs(cropped_dir, exist_ok=True)

# Inicializar el modelo PaddleOCR con parámetros ajustados
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang='en',
    drop_score=0.8
)

# Ruta del archivo de resultados
result_file = "resultados 90AMP EN TODO FILTRO v2.txt"

# Limpiar o crear el archivo de resultados
with open(result_file, "w") as f:
    f.write("Resultados de OCR:\n\n")

def extract_value_from_filename(filename):
    """Extrae el valor compuesto por las dos partes después de 'value_' en el nombre del archivo."""
    match = re.search(r'value_(\d+)_(\d+)', filename)
    if match:
        return match.group(1) + match.group(2)  # Combinar ambos grupos como un único número
    return None

def calculate_match_percentage(ocr_result, expected_value):
    """Calcula el porcentaje de coincidencia basado en caracteres correctos detectados, sin importar el orden."""
    ocr_chars = list(ocr_result)
    expected_chars = list(expected_value)
    correct_chars = 0

    # Contar los caracteres correctos (sin importar el orden)
    for char in expected_chars:
        if char in ocr_chars:
            correct_chars += 1
            ocr_chars.remove(char)

    total_chars = len(expected_chars)
    if total_chars == 0:
        return 0

    match_percentage = (correct_chars / total_chars) * 100
    return match_percentage

# Variables para calcular el porcentaje global
total_images = 0
successful_matches = 0
deficient_matches = 0
none_type_errors = 0

# Factor de ampliación del recorte
expansion_factor = 0.1

# Procesar las imágenes en el directorio
for filename in tqdm(os.listdir(image_dir)):
    image_path = os.path.join(image_dir, filename)

    # Verificar que sea una imagen
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        # Cargar la imagen
        image = cv2.imread(image_path)
        if image is None:
            print(f"No se pudo cargar la imagen: {image_path}")
            none_type_errors += 1
            continue

        # Obtener el valor esperado del nombre del archivo
        expected_value = extract_value_from_filename(filename)
        if expected_value is None:
            print(f"No se encontró un valor esperado en el archivo: {filename}")
            none_type_errors += 1
            continue

        # Realizar la predicción con YOLO
        results = model(image, conf=0.5)
        
        # Reiniciar valores para cada imagen
        numerical_results = ""
        ocr_cleaned = ""

        # Procesar cada resultado para recortar la zona de interés
        for result in results:
            masks = result.masks
            if masks:
                for i, mask in enumerate(masks.data):
                    mask = mask.cpu().numpy()
                    mask_binary = (mask * 255).astype(np.uint8)
                    mask_resized = cv2.resize(mask_binary, (image.shape[1], image.shape[0]))
                    mask_3channel = cv2.merge([mask_resized, mask_resized, mask_resized])
                    masked_image = cv2.bitwise_and(image, mask_3channel)

                    # Obtener el rectángulo delimitador y expandirlo
                    x, y, w, h = cv2.boundingRect(mask_resized)
                    
                    # Calcular el nuevo recorte con el factor de ampliación
                    x_exp = max(int(x - w * expansion_factor), 0)
                    y_exp = max(int(y - h * expansion_factor), 0)
                    w_exp = min(int(w + w * 2 * expansion_factor), image.shape[1] - x_exp)
                    h_exp = min(int(h + h * 2 * expansion_factor), image.shape[0] - y_exp)
                    
                    roi = image[y_exp:y_exp + h_exp, x_exp:x_exp + w_exp]

                    # Realizar OCR
                    ocr_result = ocr.ocr(roi)
                    for line in ocr_result[0]:
                        text = line[1][0]  # El valor OCR detectado
                        numerical_results += text

                        # Escribir las coordenadas, el valor OCR y el score en el archivo de resultados
                        with open(result_file, "a") as f:
                            coords = line[0]
                            score = line[1][1]
                            f.write(f"Coordenadas: {coords} - Valor OCR: {text} - Score: {score:.2f}\n")

                    # Limpiar el resultado OCR
                    ocr_cleaned = re.sub(r'\D', '', numerical_results)  # Eliminar caracteres no numéricos

                    # Guardar la imagen recortada
                    cropped_filename = f"{os.path.splitext(filename)[0]}_mask{i}.png"
                    cropped_path = os.path.join(cropped_dir, cropped_filename)
                    cv2.imwrite(cropped_path, roi)

        # Calcular el porcentaje de coincidencia
        match_percentage = calculate_match_percentage(ocr_cleaned, expected_value)

        # Actualizar contadores globales
        total_images += 1
        if match_percentage >= 90:
            successful_matches += 1
        else:
            deficient_matches += 1

        # Guardar el resultado específico de esta imagen
        with open(result_file, "a") as f:
            f.write(f"Archivo: {filename}\n")
            f.write(f"Valor esperado: {expected_value}\n")
            f.write(f"Valor OCR: {ocr_cleaned}\n")
            f.write(f"Porcentaje de coincidencia: {match_percentage:.2f}%\n\n")

    except Exception as e:
        none_type_errors += 1
        with open(result_file, "a") as f:
            f.write(f"Error procesando la imagen {filename}: {e}\n\n")

# Guardar el total de imágenes procesadas
with open(result_file, "a") as f:
    f.write(f"\nTotal de imágenes procesadas: {total_images}\n")
    f.write(f"Cantidad de lecturas correctas: {successful_matches}\n")
    f.write(f"Cantidad de lecturas deficientes: {deficient_matches}\n")
    f.write(f"Cantidad de errores NoneType: {none_type_errors}\n")
